# Data cleaning 

In [1]:
import pandas as pd

In [139]:
from use_cases.emotions import process_emotions
from use_cases.contributions import process_contributions
from use_cases.individuals import process_individuals
from use_cases.needs import process_needs
from use_cases.family_needs import process_needs
from use_cases.diag_members import get_dialogue_members

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data Surveys

In [6]:
chunksize   = 100 # Just for debugging
survey_path = './datos/Dialogo/BBDD_dialogos_final.csv' 
survey = pd.read_csv(survey_path, chunksize=chunksize, low_memory=False)
for survey in survey: break

In [9]:
survey.head()

,ID Archivo,Fecha,Hora Inicio,Hora Termino,Lugar,Dirección,Comuna,Region,Participantes,Grupo,...,LP_COD_NIVEL_P27,LP_COD_SEXO_P28,LP_COD_NIVEL_P28,LP_COD_SEXO_P29,LP_COD_NIVEL_P29,LP_COD_SEXO_P30,LP_COD_NIVEL_P30,Unnamed: 251,ALPDEI,TIPO INCIDECIA
0,ENC_U_4602780640112847056,25-02-2020,10:00:00,12:00:00,Municipalidad de Saavedra,Ejercito 1424,Saavedra,de la Araucania,5,Usuarios de oficina movil de bienes nacionales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Aprobada sin Incidencias
1,ENC_U_4602778880117363308,18-02-2020,11:00:00,13:00:00,Sede Comunidad Indígena Millapán Romero,Sector Coipulafquen,Padre Las Casas,de la Araucania,5,Directiva y Socios de la Comunidad Millapán Ro...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Aprobada sin Incidencias
2,ENC_U_4602777200118509403,14-02-2020,10:20:00,12:20:00,Municipalidad de cunco,Pedro aguirre cerda 600,Cunco,de la Araucania,5,Usuarios oficina movil de bienes nacionales.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Encuesta Digitada sin Incidencias
3,ENC_U_4602666791193242253,16-03-2020,15:30:00,16:30:00,INJUV,Lautaro Navarro 631,Punta Arenas,Magallanes,5,Grupo familiar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Aprobada con Incidencias
4,ENC_U_4602651983012551467,13-03-2020,15:00:00,17:20:00,CASA DE ACOGIDA COPIAPÓ,NaN,Copiapó,Atacama,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Aprobada con Incidencias


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Personal Information 

In [150]:
get_dialogue_members(survey)

[ID Archivo                            ENC_U_4602780640112847056
Fecha                                                25-02-2020
Hora Inicio                                            10:00:00
Hora Termino                                           12:00:00
Lugar                                 Municipalidad de Saavedra
Dirección                                         Ejercito 1424
Comuna                                                 Saavedra
Region                                          de la Araucania
Participantes                                                 5
Grupo            Usuarios de oficina movil de bienes nacionales
Name: 0, dtype: object, ID Archivo                            ENC_U_4602778880117363308
Fecha                                                18-02-2020
Hora Inicio                                            11:00:00
Hora Termino                                           13:00:00
Lugar                   Sede Comunidad Indígena Millapán Romero
Dirección      

### Emotions

<img src="diagrams/q1.png" width=730 height=520 />

In [127]:
%%time
emo_names, emo_exp = process_emotions(survey)

CPU times: user 753 ms, sys: 66.4 ms, total: 820 ms
Wall time: 823 ms


In [136]:
emo_names = emo_names.sample(2)
emo_names

,name,token,macro,emo_id
13,confianza,[confianza],confianza,11
51,empoderamiento,[empoderamiento],empoderamiento,119


In [137]:
emo_exp[emo_exp['emo_id'].isin(emo_names['emo_id'])]

,file_ids,text,text_tokens,emo_id
13,ENC_U_4601872630118251597,de que las demandas y manifestaciones (sanas) ...,"[demandas, manifestaciones, sanas, logren, mej...",11
51,ENC_U_4598337190995449677,- las personas comenzaron a reconocer cuales e...,"[personas, comenzaron, reconocer, cuales, dere...",119
53,ENC_U_4598232470115381386,en que chile y los chilenos siempre podemos sa...,"[chile, chilenos, siempre, podemos, salir, ade...",11


### Contributions